首先加载将要使用的软件包。

In [1]:
import pandas as pd
import numpy as np
import os
# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

In [2]:
# Display the folders and files in current directory;
import os
for dirname, _, filenames in os.walk('../dataset/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../dataset/600031.csv


add Markdown
调用pandas的函数 read_csv() 读取 csv 文件内容到DataFrame df_train。

读取数据以后，检查 df_train 的最初几行数据，同时可以看到每一列特征值。有些特征是是数值型的，有一些是字符型的。其中'Survived'是旅客的幸存标志。这一幸存标志在以后的例子中会作为训练数据的标签(the targets for training data)。

In [3]:
# Load data
df_train = pd.read_csv('../dataset/600031.csv')
# Show first lines of data
df_train.head()

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2020-11-09,27.80,27.54,28.14,26.89,856249,2348800992,4.55,0.18,0.05,1.01
1,2020-11-10,27.98,27.32,27.98,26.91,558824,1523209504,3.89,-0.80,-0.22,0.66
2,2020-11-11,27.33,28.31,28.86,27.21,1015374,2873205488,6.04,3.62,0.99,1.20
3,2020-11-12,28.31,28.70,28.88,27.91,718595,2041558432,3.43,1.38,0.39,0.85
4,2020-11-13,28.28,28.16,28.50,27.60,674807,1888773520,3.14,-1.88,-0.54,0.80


# 数据检查

df_train.shape 显示数据的行数和列数；df_info() 显示数据的基本信息，包括每一列特征值的名称，行数，数值类型，和占用的内存空间。

df_trian.desvribe()显示数据每一列的统计信息：平均值，方差，最大值，最小值，分位数，等。

In [4]:
df_train.shape

(252, 11)

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      252 non-null    object 
 1   开盘      252 non-null    float64
 2   收盘      252 non-null    float64
 3   最高      252 non-null    float64
 4   最低      252 non-null    float64
 5   成交量     252 non-null    int64  
 6   成交额     252 non-null    int64  
 7   振幅      252 non-null    float64
 8   涨跌幅     252 non-null    float64
 9   涨跌额     252 non-null    float64
 10  换手率     252 non-null    float64
dtypes: float64(8), int64(2), object(1)
memory usage: 21.8+ KB


# **缺失数据的处理**

In [6]:
# Function to check the missing percent of a DatFrame;
def check_missing_data(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False) * 100 /len(df),2)
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])

In [7]:
check_missing_data(df_train)

,Total,Percent
日期,0,0.0
开盘,0,0.0
收盘,0,0.0
最高,0,0.0
最低,0,0.0
成交量,0,0.0
成交额,0,0.0
振幅,0,0.0
涨跌幅,0,0.0
涨跌额,0,0.0


一个简单的处理缺失数据的方式，就是用0和1填补 'something'的缺失数据。因为something本身的字符没有意义，所以在 something缺失的位置填0，并且用1替代something的原有非空字符。

此外：
* 还可以用对应列的平均值填补缺失的数据。

    `#complete missing age with median
    data['Age'].fillna(data['Age'].mean(), inplace = True)`

* 用相应列中出现最多的字符来代替缺失的值，这通过 dataframe 的函数 data['Embarked'].mode() 来实现。

    `#complete missing Embarked with Mode
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace = True)`

# **数据清理**

* 检查冗余信息：相关性是检查冗余信息的有效方法。当某两列的相关系数非常高（接近1.0）的时候，我们认为这两列的信息是冗余的。以下的相关系数矩阵显示这个例子中没有冗余信息。

In [8]:
df_train.corr()

,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
开盘,1.000000,0.987127,0.994168,0.995600,-0.062605,0.309691,0.466374,-0.025143,-0.041680,-0.062120
收盘,0.987127,1.000000,0.995569,0.994422,-0.053926,0.316265,0.500237,0.121713,0.107988,-0.053420
最高,0.994168,0.995569,1.000000,0.995075,-0.028074,0.345375,0.527361,0.059532,0.042311,-0.027562
最低,0.995600,0.994422,0.995075,1.000000,-0.085634,0.286161,0.443406,0.039373,0.026148,-0.085168
成交量,-0.062605,-0.053926,-0.028074,-0.085634,1.000000,0.921587,0.530023,0.104503,0.061271,0.999993
成交额,0.309691,0.316265,0.345375,0.286161,0.921587,1.000000,0.704641,0.109323,0.061693,0.921802
振幅,0.466374,0.500237,0.527361,0.443406,0.530023,0.704641,1.000000,0.289007,0.235551,0.530612
涨跌幅,-0.025143,0.121713,0.059532,0.039373,0.104503,0.109323,0.289007,1.000000,0.979940,0.104511
涨跌额,-0.041680,0.107988,0.042311,0.026148,0.061271,0.061693,0.235551,0.979940,1.000000,0.061297
换手率,-0.062120,-0.053420,-0.027562,-0.085168,0.999993,0.921802,0.530612,0.104511,0.061297,1.000000


* 标准方差用于检测无用信息。当某一列的方差接近0.0时，这一列的所有值几乎时相同的，也就是提供了无用信息。这样的列可以被删除。我们的例子中，方差都远大于0，可以认为没有无用信息。

In [9]:
df_train.std()

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_7032/905067297.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_train.std()


开盘     5.994146e+00
收盘     6.003877e+00
最高     6.307932e+00
最低     5.695554e+00
成交量    6.820901e+05
成交额    2.159678e+09
振幅     2.011476e+00
涨跌幅    2.977870e+00
涨跌额    1.011197e+00
换手率    8.032729e-01
dtype: float64

# **非数值变量的处理**

字符串类型，我们需要将其转化为数值变量。以下的代码中，我们将用0取代和用1取代。